This post draws parallels between Python and Pyspark. If you are already adept in Python, then the move 
into Pyspark is relatively easy. Here I try to show how similar many of the key constructs are. It is important
to note that writing code in Python and in Pyspark work very differently. In Python each statement is evaluated
immediately. Pyspark, on the other hand has transformations (operations performed on dataframes etc) and actions.
Pyspark does lazy evaluation i.e. the transformations on the dataframe do not happen immediately. Spark
queues the transformations and evaluates them only when an action like show(), count(), collect() are performed.
Regardless, it is important to notice the essential similarities in coding these 2 languages namely Python and its
'big brother' Pyspark for handling small data sets to large datasets

## 1a. Read CSV file - Python
This is a common operations that will be required whether one uses Python or Pyspark. To see how load a CSV file to Databricks see this video [Upload Flat File to Databricks Table](https://www.youtube.com/watch?v=H5LxjaJgpSk)

In [3]:
import pandas as pd
import os
os.getcwd()
#Read CSV file
tendulkar = pd.read_csv("/dbfs/FileStore/tables/tendulkar.csv", header='infer')
#Check the shape of the dataframe
tendulkar.shape



Out[ 73 ]: (347, 12)

##1.b Read CSV file - Pyspark - Option 1

In [5]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext 
sql=SQLContext.getOrCreate(SparkContext.getOrCreate())
tendulkar1= (sql
         .read.format("com.databricks.spark.csv")
         .options(delimiter=',', header='true', inferschema='true')
         .load("/FileStore/tables/tendulkar.csv"))
tendulkar1.count()

Out[ 74 ]: 347

##1c. Read CSV file - Pyspark - Option 2

In [7]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Read CSV').getOrCreate()
tendulkar2 = spark.read.format('csv').option('header','true').load('/FileStore/tables/tendulkar.csv')
tendulkar2.count()

Out[ 75 ]: 347

##2a. Data frame Shape -Python
Determine the shape of the dataframe

In [9]:
tendulkar.shape

Out[ 76 ]: (347, 12)

##2b.  Data frame Shape -Pyspark
When computing the shape of a dataframe we have to do compute the count of rows and the length of the columns separately

In [11]:
tendulkar1.count()
len(tendulkar1.columns)
def dfShape(df):
  return(df.count(),len(df.columns))

dfShape(tendulkar1)

Out[ 77 ]: (347, 12)

##3a.  Data frame columns - Python

In [13]:
tendulkar.columns

Out[ 78 ]: 
Index([u'Runs', u'Mins', u'BF', u'4s', u'6s', u'SR', u'Pos', u'Dismissal',
 u'Inns', u'Opposition', u'Ground', u'Start Date'],
 dtype='object')

## 4a. Rename columns - Python

In [15]:
tendulkar.columns=[['Runs','Minutes','BallsFaced','Fours','Sixes','StrikeRate','Position','Dismissal','Innings','Opposition','Ground','StartDate']]
tendulkar.columns

Out[ 79 ]: 
Index([u'Runs', u'Minutes', u'BallsFaced', u'Fours', u'Sixes', u'StrikeRate',
 u'Position', u'Dismissal', u'Innings', u'Opposition', u'Ground',
 u'StartDate'],
 dtype='object')

##4b. Rename columns - Pyspark

In [17]:
tendulkar1 = tendulkar1.withColumnRenamed("Runs", "Runs")\
                       .withColumnRenamed("Mins", "Minutes")\
                       .withColumnRenamed("BF", "BallsFaced")\
                       .withColumnRenamed("4s", "Fours")\
                       .withColumnRenamed("6s", "Sixes")\
                       .withColumnRenamed("SR", "StrikeRate")\
                       .withColumnRenamed("Dismissal", "Dismissal")               


##5a. Dtypes - Python

In [19]:
tendulkar.dtypes

Out[ 81 ]: 
Runs object
Minutes object
BallsFaced object
Fours object
Sixes object
StrikeRate object
Position object
Dismissal object
Innings object
Opposition object
Ground object
StartDate object
dtype: object

##5b. Dtypes -Pyspark

In [21]:
tendulkar1.dtypes

Out[ 82 ]: 
[('Runs', 'string'),
 ('Minutes', 'string'),
 ('BallsFaced', 'string'),
 ('Fours', 'string'),
 ('Sixes', 'string'),
 ('StrikeRate', 'string'),
 ('Pos', 'string'),
 ('Dismissal', 'string'),
 ('Inns', 'string'),
 ('Opposition', 'string'),
 ('Ground', 'string'),
 ('Start Date', 'string')]

## 5c. printSchema

In [23]:
tendulkar1.printSchema()

root
-- Runs: string (nullable = true)
-- Minutes: string (nullable = true)
-- BallsFaced: string (nullable = true)
-- Fours: string (nullable = true)
-- Sixes: string (nullable = true)
-- StrikeRate: string (nullable = true)
-- Pos: string (nullable = true)
-- Dismissal: string (nullable = true)
-- Inns: string (nullable = true)
-- Opposition: string (nullable = true)
-- Ground: string (nullable = true)
-- Start Date: string (nullable = true)

##6a.  Select columns - Python

In [25]:
import pandas as pd
import os
os.getcwd()
#Read CSV file
tendulkar = pd.read_csv("/dbfs/FileStore/tables/tendulkar.csv", header='infer')
df=tendulkar[['Runs','Mins','BF']]
df.head(10)

Out[ 84 ]: 
 Runs Mins BF
0 15 28 24
1 DNB - -
2 59 254 172
3 8 24 16
4 41 124 90
5 35 74 51
6 57 193 134
7 0 1 1
8 24 50 44
9 88 324 266

##6b.  Select columns - Pyspark

In [27]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext 
sql=SQLContext.getOrCreate(SparkContext.getOrCreate())
tendulkar1= (sql
         .read.format("com.databricks.spark.csv")
         .options(delimiter=',', header='true', inferschema='true')
         .load("/FileStore/tables/tendulkar.csv"))
df1=tendulkar1.select('Runs','Mins','BF')
df1.show(10)

+----+----+---+
Runs|Mins| BF|
+----+----+---+
 15| 28| 24|
 DNB| -| -|
 59| 254|172|
 8| 24| 16|
 41| 124| 90|
 35| 74| 51|
 57| 193|134|
 0| 1| 1|
 24| 50| 44|
 88| 324|266|
+----+----+---+
only showing top 10 rows

##7a. Filter rows by criteria - Python

In [29]:
b = tendulkar['Runs'] >50
df = tendulkar[b]
df.head(10)

Out[ 86 ]: 
 Runs Mins BF 4s 6s SR Pos Dismissal Inns Opposition Ground \
0 15 28 24 2 0 62.5 6 bowled 2 v Pakistan Karachi 
1 DNB - - - - - - - 4 v Pakistan Karachi 
2 59 254 172 4 0 34.3 6 lbw 1 v Pakistan Faisalabad 
3 8 24 16 1 0 50 6 run out 3 v Pakistan Faisalabad 
4 41 124 90 5 0 45.55 7 bowled 1 v Pakistan Lahore 
5 35 74 51 5 0 68.62 6 lbw 1 v Pakistan Sialkot 
6 57 193 134 6 0 42.53 6 caught 3 v Pakistan Sialkot 
7 0 1 1 0 0 0 6 caught 2 v New Zealand Christchurch 
8 24 50 44 3 0 54.54 6 caught 3 v New Zealand Christchurch 
9 88 324 266 5 0 33.08 6 caught 1 v New Zealand Napier 

 Start Date 
0 15-Nov-89 
1 15-Nov-89 
2 23-Nov-89 
3 23-Nov-89 
4 1-Dec-89 
5 9-Dec-89 
6 9-Dec-89 
7 2-Feb-90 
8 2-Feb-90 
9 9-Feb-90

##7b Filtering by criteria - Pyspark

In [31]:
df1=tendulkar1.filter(tendulkar1['Runs']>50)
df1.show(10)

+----+----+---+---+---+-----+---+---------+----+--------------+------------+----------+
Runs|Mins| BF| 4s| 6s| SR|Pos|Dismissal|Inns| Opposition| Ground|Start Date|
+----+----+---+---+---+-----+---+---------+----+--------------+------------+----------+
 59| 254|172| 4| 0| 34.3| 6| lbw| 1| v Pakistan| Faisalabad| 23-Nov-89|
 57| 193|134| 6| 0|42.53| 6| caught| 3| v Pakistan| Sialkot| 9-Dec-89|
 88| 324|266| 5| 0|33.08| 6| caught| 1| v New Zealand| Napier| 9-Feb-90|
 68| 216|136| 8| 0| 50| 6| caught| 2| v England| Manchester| 9-Aug-90|
 114| 228|161| 16| 0| 70.8| 4| caught| 2| v Australia| Perth| 1-Feb-92|
 111| 373|270| 19| 0|41.11| 4| caught| 2|v South Africa|Johannesburg| 26-Nov-92|
 73| 272|208| 8| 1|35.09| 5| caught| 2|v South Africa| Cape Town| 2-Jan-93|
 165| 361|296| 24| 1|55.74| 4| caught| 1| v England| Chennai| 11-Feb-93|
 78| 285|213| 10| 0|36.61| 4| lbw| 2| v England| Mumbai| 19-Feb-93|
 62| 128|114| 7| 0|54.38| 4| caught| 1| v Zimbabwe| Delhi| 13-Mar-93|
+----+----+---+---+---+-----+---+---------+----+--------------+------------+----------+
only showing top 10 rows

##8a.  Display unique contents of a column - Python

In [33]:
tendulkar = pd.read_csv("/dbfs/FileStore/tables/tendulkar.csv", header='infer')
tendulkar['Runs'].unique()

Out[ 88 ]: 
array(['15', 'DNB', '59', '8', '41', '35', '57', '0', '24', '88', '5',
 '10', '27', '68', '119*', '21', '11', '16', '7', '40', '148*', '6',
 '17', '114', '111', '1', '73', '50', '9*', '165', '78', '62',
 'TDNB', '28', '104*', '71', '142', '96', '43', '11*', '34', '85',
 '179', '54', '4', '0*', '52*', '2', '122', '31', '177', '74', '42',
 '18', '61', '36', '169', '9', '15*', '92', '83', '143', '139', '23',
 '148', '13', '155*', '79', '47', '113', '67', '136', '29', '53',
 '124*', '126*', '44*', '217', '116', '52', '45', '97', '20', '39',
 '201*', '76', '65', '126', '36*', '69', '155', '22*', '103', '26',
 '90', '176', '117', '86', '12', '193', '16*', '51', '32', '55',
 '37', '44', '241*', '60*', '194*', '3', '32*', '248*', '94', '22',
 '109', '19', '14', '28*', '63', '64', '101', '122*', '91', '82',
 '56*', '154*', '153', '49', '10*', '103*', '160', '100*', '105*',
 '100', '106', '84', '203', '98', '38', '214', '53*', '111*', '146',
 '14*', '56', '80', '25', '81', '13*'], dtype=object)

##8b. Display unique contents of a column - Pyspark

In [35]:
from pyspark.sql.functions import *
sqlContext=SQLContext.getOrCreate(SparkContext.getOrCreate())
tendulkar1= (sqlContext
         .read.format("com.databricks.spark.csv")
         .options(delimiter=',', header='true', inferschema='true')
         .load("/FileStore/tables/tendulkar.csv"))
tendulkar1.select('Runs').rdd.distinct().collect()

Out[ 89 ]: 
[Row(Runs=u'24'),
 Row(Runs=u'148*'),
 Row(Runs=u'100*'),
 Row(Runs=u'0'),
 Row(Runs=u'16'),
 Row(Runs=u'80'),
 Row(Runs=u'146'),
 Row(Runs=u'19'),
 Row(Runs=u'96'),
 Row(Runs=u'16*'),
 Row(Runs=u'76'),
 Row(Runs=u'51'),
 Row(Runs=u'88'),
 Row(Runs=u'9'),
 Row(Runs=u'35'),
 Row(Runs=u'64'),
 Row(Runs=u'41'),
 Row(Runs=u'11'),
 Row(Runs=u'25'),
 Row(Runs=u'59'),
 Row(Runs=u'1'),
 Row(Runs=u'49'),
 Row(Runs=u'91'),
 Row(Runs=u'81'),
 Row(Runs=u'122*'),
 Row(Runs=u'32*'),
 Row(Runs=u'18'),
 Row(Runs=u'60*'),
 Row(Runs=u'126*'),
 Row(Runs=u'DNB'),
 Row(Runs=u'53*'),
 Row(Runs=u'116'),
 Row(Runs=u'34'),
 Row(Runs=u'65'),
 Row(Runs=u'10'),
 Row(Runs=u'26'),
 Row(Runs=u'13*'),
 Row(Runs=u'36*'),
 Row(Runs=u'165'),
 Row(Runs=u'TDNB'),
 Row(Runs=u'139'),
 Row(Runs=u'68'),
 Row(Runs=u'27'),
 Row(Runs=u'217'),
 Row(Runs=u'86'),
 Row(Runs=u'6'),
 Row(Runs=u'203'),
 Row(Runs=u'47'),
 Row(Runs=u'98'),
 Row(Runs=u'74'),
 Row(Runs=u'53'),
 Row(Runs=u'111'),
 Row(Runs=u'105*'),
 Row(Runs=u'179'),
 Row(Runs=u'154*'),
 Row(Runs=u'13'),
 Row(Runs=u'28*'),
 Row(Runs=u'50'),
 Row(Runs=u'248*'),
 Row(Runs=u'7'),
 Row(Runs=u'14*'),
 Row(Runs=u'214'),
 Row(Runs=u'32'),
 Row(Runs=u'44'),
 Row(Runs=u'28'),
 Row(Runs=u'56*'),
 Row(Runs=u'52*'),
 Row(Runs=u'73'),
 Row(Runs=u'52'),
 Row(Runs=u'103*'),
 Row(Runs=u'126'),
 Row(Runs=u'67'),
 Row(Runs=u'12'),
 Row(Runs=u'20'),
 Row(Runs=u'201*'),
 Row(Runs=u'177'),
 Row(Runs=u'119*'),
 Row(Runs=u'103'),
 Row(Runs=u'4'),
 Row(Runs=u'10*'),
 Row(Runs=u'92'),
 Row(Runs=u'55'),
 Row(Runs=u'84'),
 Row(Runs=u'142'),
 Row(Runs=u'31'),
 Row(Runs=u'45'),
 Row(Runs=u'44*'),
 Row(Runs=u'111*'),
 Row(Runs=u'29'),
 Row(Runs=u'0*'),
 Row(Runs=u'113'),
 Row(Runs=u'39'),
 Row(Runs=u'241*'),
 Row(Runs=u'15'),
 Row(Runs=u'21'),
 Row(Runs=u'69'),
 Row(Runs=u'117'),
 Row(Runs=u'193'),
 Row(Runs=u'176'),
 Row(Runs=u'5'),
 Row(Runs=u'100'),
 Row(Runs=u'15*'),
 Row(Runs=u'160'),
 Row(Runs=u'153'),
 Row(Runs=u'9*'),
 Row(Runs=u'79'),
 Row(Runs=u'54'),
 Row(Runs=u'85'),
 Row(Runs=u'143'),
 Row(Runs=u'124*'),
 Row(Runs=u'61'),
 Row(Runs=u'42'),
 Row(Runs=u'136'),
 Row(Runs=u'71'),
 Row(Runs=u'38'),
 Row(Runs=u'14'),
 Row(Runs=u'22'),
 Row(Runs=u'82'),
 Row(Runs=u'2'),
 Row(Runs=u'148'),
 Row(Runs=u'11*'),
 Row(Runs=u'101'),
 Row(Runs=u'94'),
 Row(Runs=u'78'),
 Row(Runs=u'57'),
 Row(Runs=u'90'),
 Row(Runs=u'37'),
 Row(Runs=u'62'),
 Row(Runs=u'43'),
 Row(Runs=u'109'),
 Row(Runs=u'104*'),
 Row(Runs=u'169'),
 Row(Runs=u'155*'),
 Row(Runs=u'194*'),
 Row(Runs=u'17'),
 Row(Runs=u'23'),
 Row(Runs=u'3'),
 Row(Runs=u'83'),
 Row(Runs=u'106'),
 Row(Runs=u'97'),
 Row(Runs=u'155'),
 Row(Runs=u'22*'),
 Row(Runs=u'56'),
 Row(Runs=u'114'),
 Row(Runs=u'122'),
 Row(Runs=u'36'),
 Row(Runs=u'63'),
 Row(Runs=u'40'),
 Row(Runs=u'8')]

##9a.  Aggregate mean, max, min - Python

In [37]:


import pandas as pd
import os
os.getcwd()
tendulkar = pd.read_csv("/dbfs/FileStore/tables/tendulkar.csv", header='infer')
tendulkar.shape
# Remove rows which have DNB
a=tendulkar.Runs !="DNB"
tendulkar=tendulkar[a]
tendulkar.shape

# Remove rows which have TDNB
b=tendulkar.Runs !="TDNB"
tendulkar=tendulkar[b]
tendulkar.shape

# Remove the '*' character
c= tendulkar.BF != "-"
tendulkar=tendulkar[c]
tendulkar.Runs= tendulkar.Runs.str.replace(r"[*]","")

#tendulkar.shape
type(tendulkar['Runs'][0])
tendulkar['Runs']=pd.to_numeric(tendulkar['Runs'])
tendulkar['BF']=pd.to_numeric(tendulkar['BF'])

# Group by ground  anc compute mean,min and max
df=tendulkar[['Runs','BF','Ground']].groupby('Ground').agg(['mean','min','max'])
df.head(10)

Out[ 90 ]: 
 Runs BF 
 mean min max mean min max
Ground 
Adelaide 32.600000 0 153 58.400000 5 205
Ahmedabad 40.125000 4 217 80.062500 3 344
Auckland 5.000000 5 5 13.000000 13 13
Bangalore 54.312500 0 214 98.687500 0 363
Birmingham 46.750000 1 122 71.500000 8 177
Bloemfontein 85.000000 15 155 109.500000 35 184
Bridgetown 26.000000 0 92 45.000000 2 147
Brisbane 7.666667 0 16 23.333333 3 42
Bulawayo 55.000000 36 74 88.500000 49 128
Cape Town 69.857143 9 169 155.285714 28 314

##9b.  Aggregate mean,min max - Pyspark

In [39]:
from pyspark.sql.functions import *
tendulkar1= (sqlContext
         .read.format("com.databricks.spark.csv")
         .options(delimiter=',', header='true', inferschema='true')
         .load("/FileStore/tables/tendulkar.csv"))
# Filter rows which have DNB
tendulkar1= tendulkar1.where(tendulkar1['Runs'] != 'DNB')
print(dfShape(tendulkar1))

# Filter rows which have TDNB
tendulkar1= tendulkar1.where(tendulkar1['Runs'] != 'TDNB')
print(dfShape(tendulkar1))

# Replace * with ""
tendulkar1 = tendulkar1.withColumn('Runs', regexp_replace('Runs', '[*]', ''))
tendulkar1.select('Runs').rdd.distinct().collect()

from pyspark.sql import functions as F

#Group by ground and compute mean, min and max
df=tendulkar1[['Runs','BF','Ground']].groupby(tendulkar1['Ground']).agg(F.mean(tendulkar1['Runs']),F.min(tendulkar1['Runs']),F.max(tendulkar1['Runs']))
df.show()

(330, 12)
(329, 12)
+-------------+-----------------+---------+---------+
 Ground| avg(Runs)|min(Runs)|max(Runs)|
+-------------+-----------------+---------+---------+
 Bangalore| 54.3125| 0| 96|
 Adelaide| 32.6| 0| 61|
Colombo (PSS)| 37.2| 14| 71|
 Christchurch| 12.0| 0| 24|
 Auckland| 5.0| 5| 5|
 Chennai| 60.625| 0| 81|
 Centurion| 73.5| 111| 36|
 Brisbane|7.666666666666667| 0| 7|
 Birmingham| 46.75| 1| 40|
 Ahmedabad| 40.125| 100| 8|
Colombo (RPS)| 143.0| 143| 143|
 Chittagong| 57.8| 101| 36|
 Cape Town|69.85714285714286| 14| 9|
 Bridgetown| 26.0| 0| 92|
 Bulawayo| 55.0| 36| 74|
 Delhi|39.94736842105263| 0| 76|
 Chandigarh| 11.0| 11| 11|
 Bloemfontein| 85.0| 15| 155|
Colombo (SSC)|77.55555555555556| 104| 8|
 Cuttack| 2.0| 2| 2|
+-------------+-----------------+---------+---------+
only showing top 20 rows

##10 Using SQL with Pyspark

In [41]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Read CSV').getOrCreate()
tendulkar1.createOrReplaceTempView("tendulkarTbl")
df = spark.sql("SELECT * FROM tendulkarTbl")
df.show(3)
df1 = spark.sql("SELECT * FROM tendulkarTbl where Ground='Karachi'")
df1.show()

+----+----+---+---+---+----+---+---------+----+----------+----------+----------+
Runs|Mins| BF| 4s| 6s| SR|Pos|Dismissal|Inns|Opposition| Ground|Start Date|
+----+----+---+---+---+----+---+---------+----+----------+----------+----------+
 15| 28| 24| 2| 0|62.5| 6| bowled| 2|v Pakistan| Karachi| 15-Nov-89|
 59| 254|172| 4| 0|34.3| 6| lbw| 1|v Pakistan|Faisalabad| 23-Nov-89|
 8| 24| 16| 1| 0| 50| 6| run out| 3|v Pakistan|Faisalabad| 23-Nov-89|
+----+----+---+---+---+----+---+---------+----+----------+----------+----------+
only showing top 3 rows

+----+----+---+---+---+-----+---+---------+----+----------+-------+----------+
Runs|Mins| BF| 4s| 6s| SR|Pos|Dismissal|Inns|Opposition| Ground|Start Date|
+----+----+---+---+---+-----+---+---------+----+----------+-------+----------+
 15| 28| 24| 2| 0| 62.5| 6| bowled| 2|v Pakistan|Karachi| 15-Nov-89|
 23| 49| 29| 5| 0|79.31| 4| bowled| 2|v Pakistan|Karachi| 29-Jan-06|
 26| 74| 47| 5| 0|55.31| 4| bowled| 4|v Pakistan|Karachi| 29-Jan-06|
+----+----+---+---+---+-----+---+---------+----+----------+-------+----------+

##11a.  Apply lambda - Python

In [43]:
import pandas as pd
import numpy as np
import os
os.getcwd()
tendulkar = pd.read_csv("/dbfs/FileStore/tables/tendulkar.csv", header='infer')
tendulkar['4s']=tendulkar['4s'].convert_objects(convert_numeric=True)
a=tendulkar['4s'].apply(lambda x: x*4)
a

/tmp/1539000111605-0/PythonShell.py:6: FutureWarning: convert_objects is deprecated. Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
 import os
 Out[ 93 ]: 
0 8.0
1 NaN
2 16.0
3 4.0
4 20.0
5 20.0
6 24.0
7 0.0
8 12.0
9 20.0
10 4.0
11 NaN
12 4.0
13 16.0
14 32.0
15 68.0
16 12.0
17 NaN
18 12.0
19 4.0
20 0.0
21 20.0
22 56.0
23 4.0
24 8.0
25 64.0
26 4.0
27 0.0
28 4.0
29 76.0
 ... 
317 20.0
318 32.0
319 0.0
320 32.0
321 16.0
322 32.0
323 36.0
324 12.0
325 4.0
326 12.0
327 8.0
328 8.0
329 12.0
330 20.0
331 8.0
332 NaN
333 4.0
334 8.0
335 52.0
336 4.0
337 0.0
338 28.0
339 0.0
340 4.0
341 20.0
342 8.0
343 20.0
344 0.0
345 8.0
346 48.0
Name: 4s, dtype: float64

##11b. Apply lambda - Pyspark

In [45]:
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType
tendulkar1= (sqlContext
         .read.format("com.databricks.spark.csv")
         .options(delimiter=',', header='true', inferschema='true')
         .load("/FileStore/tables/tendulkar.csv"))

tendulkar1 = tendulkar1.withColumn("4s", tendulkar1["4s"].cast(IntegerType()))

tendulkar2= tendulkar1.where(col("4s").isNotNull())
fours = udf(lambda x: 4*x, IntegerType())
users = tendulkar2.withColumn("4s",fours(tendulkar1["4s"]))
users.show(5)
#df_ascii = df.select('integers', spark_convert_ascii('integers').alias('ascii_map'))

+----+----+---+---+---+-----+---+---------+----+----------+----------+----------+
Runs|Mins| BF| 4s| 6s| SR|Pos|Dismissal|Inns|Opposition| Ground|Start Date|
+----+----+---+---+---+-----+---+---------+----+----------+----------+----------+
 15| 28| 24| 8| 0| 62.5| 6| bowled| 2|v Pakistan| Karachi| 15-Nov-89|
 59| 254|172| 16| 0| 34.3| 6| lbw| 1|v Pakistan|Faisalabad| 23-Nov-89|
 8| 24| 16| 4| 0| 50| 6| run out| 3|v Pakistan|Faisalabad| 23-Nov-89|
 41| 124| 90| 20| 0|45.55| 7| bowled| 1|v Pakistan| Lahore| 1-Dec-89|
 35| 74| 51| 20| 0|68.62| 6| lbw| 1|v Pakistan| Sialkot| 9-Dec-89|
+----+----+---+---+---+-----+---+---------+----+----------+----------+----------+
only showing top 5 rows

##12. Conclusion
The above post shows some of the most important equivalent constructs in Python and Pyspark. I will be adding more constructs soon.
Watch this space.